# Bake-off: Word-level entailment with neural networks

In [29]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2018"

## Contents

0. [Overview](#Overview)
0. [Set-up](#Set-up)
0. [Data](#Data)
  0. [Edge disjoint](#Edge-disjoint)
  0. [Word disjoint](#Word-disjoint)
  0. [Word disjoint and balanced](#Word-disjoint-and-balanced)
0. [Baseline](#Baseline)
  0. [Representing words: vector_func](#Representing-words:-vector_func)
  0. [Combining words into inputs: vector_combo_func](#Combining-words-into-inputs:-vector_combo_func)
  0. [Classifier model](#Classifier-model)
  0. [Baseline results](#Baseline-results)
0. [Bake-off submission](#Bake-off-submission)

## Overview

__Problem__: Word-level natural language inference.

Training examples are pairs of words $(w_{L}, w_{R}), y$ with $y$ a relation in

* __synonym__: very roughly identical meanings; symmetric
* __hyponym__: e.g., _puppy_ is a hyponym of _dog_
* __hypernym__:  e.g., _dog_ is a hypernym of _puppy_
* __antonym__: semantically opposed within a domain; symmetric

The dataset is due to [Bowman et al. 2015](https://arxiv.org/abs/1406.1827). See [below](#Data) for details on how it was processed for this bake-off.

## Set-up

0. Make sure your environment includes all the requirements for [the cs224u repository](https://github.com/cgpotts/cs224u/).

0. Make sure you have the [the Wikipedia 2014 + Gigaword 5 distribution](http://nlp.stanford.edu/data/glove.6B.zip) of  pretrained GloVe vectors downloaded and unzipped, and that `glove_home` below is pointing to it.

0. Make sure `wordentail_filename` below is pointing to the full path for `nli_wordentail_bakeoff_data.json`, which is included in [the nlidata.zip archive](http://web.stanford.edu/class/cs224u/data/nlidata.zip).

In [30]:
from collections import defaultdict
import json
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tf_shallow_neural_classifier import TfShallowNeuralClassifier
import nli
import utils

In [31]:
nlidata_home = 'nlidata'

wordentail_filename = os.path.join(
    nlidata_home, 'nli_wordentail_bakeoff_data.json')

glove_home = os.path.join("vsmdata", "glove.6B")

## Data

As noted above, the dataset was originally released by [Bowman et al. 2015](https://arxiv.org/abs/1406.1827), who derived it from [WordNet](https://wordnet.princeton.edu) using some heuristics (and thus it might contain some errors or unintuitive pairings).

I've processed the data into three different train/test splits, in an effort to put some pressure on our models to actually learn these semantic relations, as opposed to exploiting regularities in the sample.

* `edge_disjoint`: The `train` and `dev` __edge__ sets are disjoint, but many __words__ appear in both `train` and `dev`.
* `word_disjoint`: The `train` and `dev` __vocabularies are disjoint__, and thus the edges are disjoint as well.
* `word_disjoint_balanced`: Like `word_disjoint`, but with each word appearing at most one time as the left word and at most one time on the right for a given relation type.

These are progressively harder problems

* For `word_disjoint`, there is real pressure on the model to learn abstract relationships, as opposed to memorizing properties of individual words.

* For `word_disjoint_balanced`, the model can't even learn that some terms tend to appear more on the left or the right. This might be a step too far. For example, appearing more on the right for `hypernym` corresponds in a deep way with being a more general term, which is a non-trivial lexical property that we want our models to learn.

In [32]:
with open(wordentail_filename) as f:
    wordentail_data = json.load(f)

The outer keys are the three splits plus a list giving the vocabulary for the entire dataset:

In [33]:
wordentail_data.keys()

dict_keys(['edge_disjoint', 'vocab', 'word_disjoint', 'word_disjoint_balanced'])

### Edge disjoint

In [34]:
wordentail_data['edge_disjoint'].keys()

dict_keys(['dev', 'train'])

This is what the split looks like; all three have this same format:

In [35]:
wordentail_data['edge_disjoint']['dev'][: 5]

[[['archived', 'records'], 'synonym'],
 [['stage', 'station'], 'synonym'],
 [['engineers', 'design'], 'hypernym'],
 [['save', 'book'], 'hypernym'],
 [['match', 'supply'], 'hypernym']]

Let's test to make sure no edges are shared between `train` and `dev`:

In [36]:
nli.get_edge_overlap_size(wordentail_data, 'edge_disjoint')

0

As we expect, a *lot* of vocabulary items are shared between `train` and `dev`:

In [37]:
nli.get_vocab_overlap_size(wordentail_data, 'edge_disjoint')

4769

This is a large percentage of the entire vocab:

In [38]:
len(wordentail_data['vocab'])

6560

Here's the distribution of labels in the `train` set. It's highly imbalanced, which will pose a challenge. (I'll go ahead and reveal that the `dev` set is similarly distributed.)

In [39]:
def label_distribution(split):
    return pd.DataFrame(wordentail_data[split]['train'])[1].value_counts()

In [40]:
label_distribution('edge_disjoint')

synonym     8865
hypernym    6475
hyponym     1044
antonym      629
Name: 1, dtype: int64

### Word disjoint

In [41]:
wordentail_data['word_disjoint'].keys()

dict_keys(['dev', 'train'])

In the `word_disjoint` split, no __words__ are shared between `train` and `dev`:

In [42]:
nli.get_vocab_overlap_size(wordentail_data, 'word_disjoint')

0

Because no words are shared between `train` and `dev`, no edges are either:

In [43]:
nli.get_edge_overlap_size(wordentail_data, 'word_disjoint')

0

The label distribution is similar to that of `edge_disjoint`, though the overall number of examples is a bit smaller:

In [44]:
label_distribution('word_disjoint')

synonym     5610
hypernym    3993
hyponym      627
antonym      386
Name: 1, dtype: int64

There is still an important bias in the data: some words appear much more often than others, and in specific positions. For example, the very general term `part` appears on the right in a large number of cases, many of them `hypernym`.

In [45]:
[[ex, y] for ex, y in wordentail_data['word_disjoint']['train'] 
 if ex[1] == 'part']

[[['frames', 'part'], 'hypernym'],
 [['heaven', 'part'], 'hypernym'],
 [['pan', 'part'], 'synonym'],
 [['middle', 'part'], 'hypernym'],
 [['shared', 'part'], 'synonym'],
 [['shares', 'part'], 'synonym'],
 [['ended', 'part'], 'hypernym'],
 [['twin', 'part'], 'synonym'],
 [['meal', 'part'], 'synonym'],
 [['bit', 'part'], 'hypernym'],
 [['sections', 'part'], 'synonym'],
 [['capacity', 'part'], 'hypernym'],
 [['beginning', 'part'], 'hypernym'],
 [['divorce', 'part'], 'hypernym'],
 [['paradise', 'part'], 'hypernym'],
 [['ends', 'part'], 'hypernym'],
 [['reduced', 'part'], 'hypernym'],
 [['units', 'part'], 'hypernym'],
 [['corner', 'part'], 'hypernym'],
 [['air', 'part'], 'hypernym'],
 [['section', 'part'], 'synonym'],
 [['something', 'part'], 'synonym'],
 [['reduce', 'part'], 'hypernym'],
 [['some', 'part'], 'synonym'],
 [['heavy', 'part'], 'hypernym'],
 [['segment', 'part'], 'hypernym'],
 [['share', 'part'], 'synonym'],
 [['hat', 'part'], 'hypernym'],
 [['maria', 'part'], 'hypernym'],
 [['

These tabulations suggest that a classifier could do well just by learning where words tend to appear:

In [46]:
def count_label_position_instances(split, pos=0):
    examples = wordentail_data[split]['train']    
    return pd.Series([(ex[pos], label) for ex, label in examples]).value_counts()

In [47]:
count_label_position_instances('word_disjoint', pos=0).head()

(forms, hypernym)       9
(tested, hypernym)      8
(questions, synonym)    8
(have, synonym)         8
(files, synonym)        8
dtype: int64

In [48]:
count_label_position_instances('word_disjoint', pos=1).head()

(be, hypernym)        51
(take, hypernym)      39
(alter, hypernym)     38
(person, hypernym)    33
(modify, hypernym)    32
dtype: int64

### Word disjoint and balanced

To see how much our models are leveraging the uneven distribution of words across the left and right positions, we also have a split in which each word $w$ appears in at most one item $((w, w_{R}), y)$ and at most one item $((w_{L}, w), y)$.

The following tests establish that the dataset has the desired properties:

In [49]:
wordentail_data['word_disjoint_balanced'].keys()

dict_keys(['dev', 'train'])

In [50]:
nli.get_edge_overlap_size(wordentail_data, 'word_disjoint_balanced')

0

In [51]:
nli.get_vocab_overlap_size(wordentail_data, 'word_disjoint_balanced')

0

In [52]:
[[ex, y] for ex, y in wordentail_data['word_disjoint_balanced']['train'] 
 if ex[1] == 'part']

[[['frames', 'part'], 'hypernym'], [['pan', 'part'], 'synonym']]

In [53]:
count_label_position_instances('word_disjoint_balanced', pos=0).head()

(milf, antonym)           1
(ended, hypernym)         1
(experiences, synonym)    1
(belief, synonym)         1
(console, synonym)        1
dtype: int64

In [54]:
count_label_position_instances('word_disjoint_balanced', pos=1).head()

(general, synonym)      1
(struggle, hypernym)    1
(blow, hyponym)         1
(port, synonym)         1
(spin, hypernym)        1
dtype: int64

## Baseline

Even in deep learning, __feature representation is the most important thing and requires care!__
For our task, feature representation has two parts: representing the individual words and combining those representations into a single network input.

### Representing words: vector_func

Let's consider two baseline word representations methods:

1. Random vectors (as returned by `utils.randvec`).
1. 50-dimensional GloVe representations.

In [55]:
def randvec(w, n=50, lower=-1.0, upper=1.0):
    """Returns a random vector of length `n`. `w` is ignored."""
    return utils.randvec(n=n, lower=lower, upper=upper)

In [56]:
# Any of the files in glove.6B will work here:
glove50_src = os.path.join(glove_home, 'glove.6B.50d.txt')

# Creates a dict mapping strings (words) to GloVe vectors:
GLOVE50 = utils.glove2dict(glove50_src)

def glove50vec(w):    
    """Return `w`'s GloVe representation if available, else return 
    a random vector."""
    return GLOVE50.get(w, randvec(w, n=50))

### Combining words into inputs: vector_combo_func

Here we decide how to combine the two word vectors into a single representation. In more detail, where `u` is a vector representation of the left word and `v` is a vector representation of the right word, we need a function `vector_combo_func` such that `vector_combo_func(u, v)` returns a new input vector `z` of dimension `m`. A simple example is concatenation:

In [57]:
def vec_concatenate(u, v):
    """Concatenate np.array instances `u` and `v` into a new np.array"""
    return np.concatenate((u, v))

`vector_combo_func` could instead be vector average, vector difference, etc. (even combinations of those) – there's lots of space for experimentation here.

### Classifier model

For a baseline model, I chose `TfShallowNeuralClassifier` with a pretty large hidden layer and a correspondingly high number of iterations. 

In [58]:
net = TfShallowNeuralClassifier(hidden_dim=200, max_iter=500)

### Baseline results

The following puts the above pieces together, using `vector_func=glove50vec`, since `vector_func=randvec` seems so hopelessly misguided for `word_disjoint` and `word_disjoint_balanced`!

First, we build the dataset:

In [59]:
X = nli.build_bakeoff_dataset(
    wordentail_data, 
    vector_func=glove50vec,
    vector_combo_func=vec_concatenate)

And then we run the experiment with `nli.bakeoff_experiment`. This trains and tests on all three splits, and additionally trains on `word_disjoint`'s `train` portion and tests on `word_disjoint_balanced`'s `dev` portion, to see what distribution of examples is more effective for this balanced evaluation.

Since the bake-off focus is `word_disjoint`, you might want to run just that evaluation. To to that, use:

In [60]:
nli.bakeoff_experiment(X, net, conditions=['word_disjoint'])

Iteration 500: loss: 9.7595701217651375

word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       150
   hypernym       0.55      0.41      0.47      1594
    hyponym       0.27      0.01      0.03       275
    synonym       0.58      0.79      0.67      2229

avg / total       0.53      0.57      0.53      4248



/Users/sherryruan/github/cs224u/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


This will run the complete evaluation:

In [61]:
nli.bakeoff_experiment(X, net)

Iteration 500: loss: 9.8007248044013989

edge_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       392
   hypernym       0.57      0.46      0.51      4310
    hyponym       0.47      0.04      0.07       710
    synonym       0.59      0.78      0.67      5930

avg / total       0.56      0.59      0.55     11342


Iteration 500: loss: 3.0768357515335083


word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       150
   hypernym       0.53      0.43      0.47      1594
    hyponym       0.18      0.01      0.01       275
    synonym       0.58      0.77      0.66      2229

avg / total       0.52      0.57      0.53      4248


Iteration 500: loss: 9.7521991133689888


word_disjoint_balanced
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       115
   hypernym       0.49      0.29      0.36       511
    hyponym       0.00      0.00      0.00       118
    synonym       0.55      0.85      0.67       831

avg / total       0.45      0.54      0.47      1575

word_disjoint_balanced, training on word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       115
   hypernym       0.48      0.39      0.43       511
    hyponym       0.17      0.01      0.02       118
    synonym       0.57      0.80      0.67       831

avg / total       0.47      0.55      0.49      1575



## Bake-off submission

__The goal__: achieve the highest average F1 score on __word_disjoint__.

__Submit:__

* Your score on the `word_disjoint` split.
* A description of the method you used: 
   * Your approach to representing words.
   * Your approach to combining them into inputs.
   * The model you used for predictions.
   
__Submission URL__: https://goo.gl/forms/CizXwS3kfPjsThxA3   

__Notes:__

* For the methods, the only requirement is that they differ in some way from the baseline above. They don't have to be completely different, though. For example, you might want to stick with the model but represent examples differently, or the reverse.

* You must train only on the `train` split. No outside training instances can be brought in. You can, though, bring in outside information via your input vectors, as long as this information is not from `dev` or `edge_disjoint`. 

* You can also augment your training data. For example, if `((A, B), synonym)` is a training instance, then so should be `((B, A), synonym)`. Similarly, `((A, B), hyponym)`  and `((B, C), hyponym)` are training cases, then so should be `((A, C), hyponym)`.

* Since the evaluation is for `word_disjoint`, you're not going to get very far with random input vectors! A GloVe featurizer is defined above. Feel free to look around for new word vectors on the Web, or even train your own using our VSM notebooks.

* You're not required to stick to `TfShallowNeuralNetwork`. For instance, you could create deeper feed-forward networks, change how they optimize, etc. As long as you have `fit` and `predict` methods with the same input and output types as our networks, you should be able to use `bakeoff_experiment`. For notes on how to extend the TensorFlow models included in this repository, see [tensorflow_models.ipynb](tensorflow_models.ipynb).

In [103]:
glove300_src = '/Users/sherryruan/data/glove/glove.6B/glove.6B.300d.txt'

# Creates a dict mapping strings (words) to GloVe vectors:
GLOVE300 = utils.glove2dict(glove300_src)

def glove300vec(w):    
    """Return `w`'s GloVe representation if available, else return 
    a random vector."""
    return GLOVE300.get(w, randvec(w, n=300))

In [118]:
glove100_src = '/Users/sherryruan/data/glove/glove.6B/glove.6B.100d.txt'

# Creates a dict mapping strings (words) to GloVe vectors:
GLOVE100 = utils.glove2dict(glove100_src)

def glove100vec(w):    
    """Return `w`'s GloVe representation if available, else return 
    a random vector."""
    return GLOVE100.get(w, randvec(w, n=100))

In [130]:
def vec_sum(u, v):
    return np.add(u, v)

In [128]:
def vec_diff(u, v):
    return np.subtract(u, v)

In [142]:
def vec_combo(u,v):
    return np.concatenate((u,v,np.subtract(u,v)))

In [87]:
# from tf_rnn_classifier import TfRNNClassifier
# vocab = wordentail_data['vocab']
# vocab.append('$UNK')
# '$UNK' in vocab
# RNNnet = TfRNNClassifier(vocab, hidden_dim=100, max_iter=100, eta=0.1)

In [91]:
class TfShallowNeuralClassifierWithDropout(TfShallowNeuralClassifier):
    def __init__(self, hidden_dim=50, keep_prob=0.8, **kwargs):
        self.hidden_dim = hidden_dim
        self.keep_prob = keep_prob
        super(TfShallowNeuralClassifierWithDropout, self).__init__(**kwargs)        
                    
    def build_graph(self):
        # All the parameters of `TfShallowNeuralClassifier`:
        self.define_parameters()
        
        # Same hidden layer:
        self.hidden = tf.nn.relu(
            tf.matmul(self.inputs, self.W_xh) + self.b_h)
        
        # Drop-out on the hidden layer:
        self.tf_keep_prob = tf.placeholder(tf.float32)
        dropout_layer = tf.nn.dropout(self.hidden, self.tf_keep_prob)
        
        # `dropout_layer` instead of `hidden` to define full model:
        self.model = tf.matmul(dropout_layer, self.W_hy) + self.b_y            
                
    def train_dict(self, X, y):
        return {self.inputs: X, self.outputs: y, 
                self.tf_keep_prob: self.keep_prob}
    
    def test_dict(self, X):
        # No dropout at test-time, hence `self.tf_keep_prob: 1.0`:
        return {self.inputs: X, self.tf_keep_prob: 1.0}

In [143]:
DropOutnet = TfShallowNeuralClassifierWithDropout(hidden_dim=200, keep_prob=0.9, max_iter=1000)

In [144]:
myX = nli.build_bakeoff_dataset(
      wordentail_data, 
      vector_func=glove100vec,
      vector_combo_func=vec_combo)

In [145]:
nli.bakeoff_experiment(myX, DropOutnet, conditions=['word_disjoint'])

Iteration 1000: loss: 7.469734847545624

word_disjoint
             precision    recall  f1-score   support

    antonym       0.00      0.00      0.00       150
   hypernym       0.54      0.46      0.49      1594
    hyponym       0.42      0.06      0.10       275
    synonym       0.59      0.76      0.66      2229

avg / total       0.54      0.57      0.54      4248



F1 Results:
concatenate:
glove 100, hidden_dim=200, keep_prob=0.8, max_iter=500: 0.54
glove 100, hidden_dim=200, keep_prob=0.9, max_iter=1000: 0.55
glove 100, hidden_dim=200, keep_prob=0.9, max_iter=2000: 0.55
glove 300, hidden_dim=500, keep_prob=0.8, max_iter=500: 0.53
glove 300, hidden_dim=500, keep_prob=0.8, max_iter=1000: 0.53
glove 300, hidden_dim=500, keep_prob=1.0, max_iter=1000: 0.53
glove 300, hidden_dim=300, keep_prob=1.0, max_iter=2000: 0.53

sum:
glove 100, hidden_dim=200, keep_prob=0.9, max_iter=2000: 0.52
glove 100, hidden_dim=200, keep_prob=1.0, max_iter=500: 0.52

diff:
glove 100, hidden_dim=200, keep_prob=0.9, max_iter=2000: 0.52

combo:
glove 100, hidden_dim=200, keep_prob=0.9, max_iter=1000: 0.54